In [ ]:
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import re
import spacy

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 552kB 22.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=3742beee3061af91647ab5d0b7113c719a9136790fd36338b4da44151ddd8f56
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=434b4e66fce938495147d33b1c928d7d7a45965631851d0e5df2918249b01e37
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/BE/Sem 8/ML/IA-2/emails.csv' 
emails = pd.read_csv(path)
emails.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
emails.shape

(517401, 2)

In [ ]:
# email_subset = emails
email_subset = emails[:50000]
print(email_subset.shape)
print(email_subset.head())

(50000, 2)
                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }

In [ ]:
# cleaning 
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [ ]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [ ]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body  ...                    from_
0                               Here is our forecast  ...  phillip.allen@enron.com
1  Traveling to have a business meeting takes the...  ...  phillip.allen@enron.com
2                     test successful.  way to go!!!  ...  phillip.allen@enron.com
3  Randy,Can you send me a schedule of the salary...  ...  phillip.allen@enron.com
4                                                     ...  phillip.allen@enron.com

[5 rows x 3 columns]


In [ ]:
email_df.shape


(50000, 3)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(email_df.iloc[1]['body'])

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.


In [ ]:
data = email_df.body.values.tolist()
type(data)

list

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
data_words = list(sent_to_words(data))

In [ ]:
print(data_words[3])

['randy', 'can', 'you', 'send', 'me', 'schedule', 'of', 'the', 'salary', 'and', 'level', 'of', 'everyone', 'in', 'thescheduling', 'group', 'plus', 'your', 'thoughts', 'on', 'any', 'changes', 'that', 'need', 'to', 'be', 'made', 'patti', 'for', 'example', 'phillip']


In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [ ]:
print(trigram_mod[bigram_mod[data_words[200]]])

['forwarded', 'by', 'phillip_allen', 'hou', 'ect', 'on', 'hello', 'men', 'have', 'attached', 'my', 'worksheet', 'in', 'case', 'you', 'want', 'to', 'review', 'the', 'data', 'while_amon_holiday', 'thanks']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

In [ ]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(data_lemmatized[200])

['ect', 'man', 'attach', 'want', 'datum', 'thank']


In [ ]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
print(lda_model.print_topics())

[(0, '0.072*"document" + 0.045*"transaction" + 0.032*"attorney" + 0.025*"color" + 0.025*"contractor" + 0.025*"width" + 0.024*"board" + 0.022*"align" + 0.021*"font" + 0.020*"tr"'), (1, '0.057*"state" + 0.021*"status" + 0.021*"bill" + 0.021*"public" + 0.020*"vote" + 0.019*"release" + 0.019*"termination" + 0.017*"agency" + 0.013*"legislation" + 0.012*"authorize"'), (2, '0.061*"meeting" + 0.039*"hold" + 0.029*"play" + 0.026*"soon" + 0.025*"speak" + 0.023*"suggest" + 0.023*"prepare" + 0.022*"final" + 0.022*"pass" + 0.019*"purpose"'), (3, '0.039*"image" + 0.038*"employer" + 0.036*"employment" + 0.028*"union" + 0.023*"increase" + 0.015*"relate" + 0.015*"accord" + 0.015*"performance" + 0.015*"announce" + 0.014*"move"'), (4, '0.041*"cash" + 0.038*"labor" + 0.029*"case" + 0.022*"cost" + 0.018*"corporate" + 0.017*"amount" + 0.014*"grant" + 0.014*"identify" + 0.013*"term" + 0.012*"value"'), (5, '0.058*"communication" + 0.056*"draft" + 0.052*"ect" + 0.050*"stock" + 0.036*"response" + 0.032*"vacatio